In [40]:
import os.path
from collections import defaultdict
import contextlib
import json
import statistics
from collections import Counter
from operator import itemgetter

import matplotlib
from matplotlib import pyplot as plt
import numpy as np
from tqdm import tqdm

from orthologue_analysis.orthogroups import init_orthogroup_df
from orthologue_analysis.species import MansoniCladeFromTool, SpeciesList
from orthologue_analysis.utils import SequenceIDMapping, orthofinder_paths
from reannotation.analysis import (
    interpro_accessions_frequently_missed_by_all_tools,
    interpro_accessions_frequently_missed_by_each_tool,
    interpro_accessions_in_novel_transcripts,
    interpro_accessions_in_missed_transcripts,
    missed_transcripts_with_significantly_more_frequent_accessions
)
from reannotation.constants import ASPARTIC_PEPTIDASE_SUPERFAMILY
from reannotation.pipelines import (
    interpro_accession_pipeline_all_tools,
    interpro_accession_pipeline,
    suspicious_orthologue_pipeline,
    novel_orthologue_pipeline
)
from reannotation.statistics import fisher_exact_for_two_lists_of_accessions, count_transcripts_with_accession
from reannotation.utils import extract_accessions_from_transcript
from utils.esm import extract_esm_means
from utils.generic import flatten_nested_dict, flatten_list_to_set
from utils.gffutils import init_db

matplotlib.use("TkAgg")


results_label = "Results_Aug05"
wbps_ann_path = "data/from_WBPS/schistosoma_mansoni.PRJEA36577.WBPS19.annotations.gff3"
braker_path = "data/from_MARS/Schistosoma_mansoni_braker3_full.gff3"
helixer_path = "data/from_MARS/Schistosoma_mansoni_helixer_full.gff3"
anno_path = "data/from_EBI/schistosoma_mansoni_gca000000000v1.gff3"
db = init_db(wbps_ann_path, "db/Sman_wbps.db")
of = orthofinder_paths(results_label, subdir="Orthogroups")

wbps_col = "Sman_LT"
braker_col = "Sman_braker3_LT"
helixer_col = "Sman_helixer_LT"
anno_col = "Sman_anno_LT"

og_df = init_orthogroup_df(of["orthogroups"])
no_og_df = init_orthogroup_df(of["orthogroups_unassigned_genes"])
seq_id_map = SequenceIDMapping(of["wd"])
mars_data_dir = os.path.join("data", "from_MARS", "")
ebi_data_dir = os.path.join("data", "from_EBI", "")

species_list = SpeciesList([
    MansoniCladeFromTool("mansoni", data_dir=mars_data_dir, data_label="Sman_LT", prot_filename_suffix=".fa"),
    MansoniCladeFromTool("mansoni_braker3_reann", data_dir=mars_data_dir, data_label="Sman_braker3_LT", prot_filename_suffix=".fa"),
    MansoniCladeFromTool("mansoni_helixer_reann", data_dir=mars_data_dir, data_label="Sman_helixer_LT", prot_filename_suffix=".fa"),
    MansoniCladeFromTool("mansoni_anno_reann", data_dir=ebi_data_dir, data_label="Sman_anno_LT", prot_filename_suffix=".fa")],
    wd_path=of["wd"],
    load_blast=True
)

wbps_species = species_list.get_species_with_data_label(wbps_col)
braker_species = species_list.get_species_with_data_label(braker_col)
helixer_species = species_list.get_species_with_data_label(helixer_col)
anno_species = species_list.get_species_with_data_label(anno_col)

with open("data/acc_product.json", "r") as f:
    acc_product = json.loads(f.read())

min_freq = 10

interproscan_dir = "data/from_MARS/interproscan/sman"



loading /home/will/wbp_scratch/data/from_MARS/OrthoFinder/WorkingDirectory/Results_Aug05/Blast0_1.txt...
loading /home/will/wbp_scratch/data/from_MARS/OrthoFinder/WorkingDirectory/Results_Aug05/Blast0_2.txt...
loading /home/will/wbp_scratch/data/from_MARS/OrthoFinder/WorkingDirectory/Results_Aug05/Blast0_3.txt...
loading /home/will/wbp_scratch/data/from_MARS/OrthoFinder/WorkingDirectory/Results_Aug05/Blast0_2.txt...
loading /home/will/wbp_scratch/data/from_MARS/OrthoFinder/WorkingDirectory/Results_Aug05/Blast0_3.txt...
loading /home/will/wbp_scratch/data/from_MARS/OrthoFinder/WorkingDirectory/Results_Aug05/Blast0_1.txt...


# General Statistics

In [2]:
from utils.generic import makedirs
plot_dir = "plots/reannotation/"
makedirs(plot_dir)

In [3]:
print("Shared orthologues with WBPS:")
print("WBPS: {}".format(len(og_df[~og_df[wbps_col].isna()])))
print("BRAKER3: {}".format(len(og_df[~og_df[wbps_col].isna() & ~og_df[braker_col].isna()])))
print("Helixer: {}".format(len(og_df[~og_df[wbps_col].isna() & ~og_df[helixer_col].isna()])))
print("Anno: {}".format(len(og_df[~og_df[wbps_col].isna() & ~og_df[anno_col].isna()])))

Shared orthologues with WBPS:
WBPS: 8651
BRAKER3: 8003
Helixer: 8029
Anno: 8170


In [4]:
print("Total transcripts that have an orthologue with WBPS:")
print("WBPS: {}".format(og_df[~og_df[wbps_col].isna()][wbps_col].str.split(",").str.len().sum()))
print("BRAKER3: {}".format(og_df[~og_df[wbps_col].isna() & ~og_df[braker_col].isna()][braker_col].str.split(",").str.len().sum()))
print("Helixer: {}".format(og_df[~og_df[wbps_col].isna() & ~og_df[helixer_col].isna()][helixer_col].str.split(",").str.len().sum()))
print("Anno: {}".format(og_df[~og_df[wbps_col].isna() & ~og_df[anno_col].isna()][anno_col].str.split(",").str.len().sum()))

Total transcripts that have an orthologue with WBPS:
WBPS: 9427
BRAKER3: 8584
Helixer: 8737
Anno: 10078


In [5]:
print("Unique orthologues:")
wbps_na = og_df[wbps_col].isna()
braker_na = og_df[braker_col].isna()
helixer_na = og_df[helixer_col].isna()
anno_na = og_df[anno_col].isna()
novel_ogs = og_df[wbps_na & (~braker_na | ~helixer_na | ~anno_na)]
print(f"WBPS only: {len(og_df[~wbps_na & braker_na & helixer_na & anno_na])}")
print(f"BRAKER only: {len(og_df[wbps_na & ~braker_na & helixer_na & anno_na])}")
print(f"BRAKER + Helixer: {len(og_df[wbps_na & ~braker_na & ~helixer_na & anno_na])}")
print(f"BRAKER + Helixer + Anno: {len(og_df[wbps_na & ~braker_na & ~helixer_na & ~anno_na])}")
print(f"Helixer only: {len(og_df[wbps_na & braker_na & ~helixer_na & anno_na])}")
print(f"Helixer + Anno: {len(og_df[wbps_na & braker_na & ~helixer_na & ~anno_na])}")
print(f"Anno only: {len(og_df[wbps_na & braker_na & helixer_na & ~anno_na])}")
print(f"Anno + BRAKER: {len(og_df[wbps_na & ~braker_na & helixer_na & ~anno_na])}")
print(f"Any automated tool only: {len(novel_ogs)}")

Unique orthologues:
WBPS only: 17
BRAKER only: 19
BRAKER + Helixer: 51
BRAKER + Helixer + Anno: 56
Helixer only: 24
Helixer + Anno: 126
Anno only: 100
Anno + BRAKER: 95
Any automated tool only: 471


In [6]:
def count_all_mRNA_exons(label):
    count = []
    sp = species_list.get_species_with_data_label(label)
    for t in sp.db.all_features(featuretype="mRNA"):
        count.append(len(list(sp.db.children(t, featuretype="exon"))))
    return count


exon_count = defaultdict(list)
exon_count["wbps"] = count_all_mRNA_exons(wbps_col)
exon_count["anno"] = count_all_mRNA_exons(anno_col)
exon_count["braker"] = count_all_mRNA_exons(braker_col)
exon_count["helixer"] = count_all_mRNA_exons(helixer_col)

In [7]:
def count_all_mRNA_amino_acids(label):
    count = []
    sp = species_list.get_species_with_data_label(label)
    for t in sp.db.all_features(featuretype="mRNA"):
        cds_exons = list(sp.db.children(t, featuretype="CDS"))
        count.append(sp.get_amino_acid_count(cds_exons))
    return count

protein_lengths = defaultdict(list)
protein_lengths["wbps"] = count_all_mRNA_amino_acids(wbps_col)
protein_lengths["anno"] = count_all_mRNA_amino_acids(anno_col)
protein_lengths["braker"] = count_all_mRNA_amino_acids(braker_col)
protein_lengths["helixer"] = count_all_mRNA_amino_acids(helixer_col)

In [10]:
def boxplot_axes(idx, label, data, ylim=(0, 25)):
    ax = plt.subplot(1, 4, idx)
    ax.boxplot(data[label])
    if ylim:
        ax.set_ylim(top=ylim[1], bottom=ylim[0])
    ax.grid(which="both")
    ax.set_title(label.upper())
    ax.set_xticklabels([])
    ax.set_xlabel(round(statistics.mean(data[label]), 2))

boxplot_axes(1, "wbps", exon_count)
boxplot_axes(2, "anno", exon_count)
boxplot_axes(3, "braker", exon_count)
boxplot_axes(4, "helixer", exon_count)
plt.suptitle("Exon counts")
plt.text(-1.05, -1.7, 'Means')
plt.savefig(os.path.join(plot_dir, "sman_exon_count_boxplots.png"))
plt.show()

In [11]:
boxplot_axes(1, "wbps", protein_lengths, ylim=(0, 3000))
boxplot_axes(2, "anno", protein_lengths, ylim=(0, 3000))
boxplot_axes(3, "braker", protein_lengths, ylim=(0, 3000))
boxplot_axes(4, "helixer", protein_lengths, ylim=(0, 3000))
plt.suptitle("Amino acid counts")
plt.text(-1.05, -1.7, 'Means')
plt.savefig(os.path.join(plot_dir, "sman_prot_length_boxplots.png"))
plt.show()

In [13]:
wbps_species = species_list.get_species_with_data_label(wbps_col)
anno_species = species_list.get_species_with_data_label(anno_col)
braker_species = species_list.get_species_with_data_label(braker_col)
helixer_species = species_list.get_species_with_data_label(helixer_col)
x1, x2, x3 = [], [], []
x1e, x2e, x3e = [], [], []
y1, y2, y3 = [], [], []
y1e, y2e, y3e = [], [], []
for _, row in og_df[~og_df[wbps_col].isna()].iterrows():
    wbps_tran = wbps_species.db["transcript:" + list(map(str.strip, row[wbps_col].split(",")))[0].split("transcript_")[1]]
    wbps_cds_exons = list(wbps_species.db.children(wbps_tran, featuretype="CDS"))
    wbps_prot_length = wbps_species.get_amino_acid_count(wbps_cds_exons)
    if not row[anno_col] is np.nan:
        x1.append(wbps_prot_length)
        x1e.append(len(wbps_cds_exons))
        anno_tran = anno_species.db["transcript:" + list(map(str.strip, row[anno_col].split(",")))[0].split("transcript_")[1]]
        cds_exons = list(anno_species.db.children(anno_tran, featuretype="CDS"))
        y1.append(anno_species.get_amino_acid_count(cds_exons))
        y1e.append(len(cds_exons))
    if not row[braker_col] is np.nan:
        x2.append(wbps_prot_length)
        x2e.append(len(wbps_cds_exons))
        braker_tran = braker_species.db[list(map(str.strip, row[braker_col].split(",")))[0]]
        cds_exons = list(braker_species.db.children(braker_tran, featuretype="CDS"))
        y2.append(braker_species.get_amino_acid_count(cds_exons))
        y2e.append(len(cds_exons))
    if not row[helixer_col] is np.nan:
        x3.append(wbps_prot_length)
        x3e.append(len(wbps_cds_exons))
        helixer_tran = helixer_species.db[list(map(str.strip, row[helixer_col].split(",")))[0]]
        cds_exons = list(helixer_species.db.children(helixer_tran, featuretype="CDS"))
        y3.append(helixer_species.get_amino_acid_count(cds_exons))
        y3e.append(len(cds_exons))

In [14]:
def scatter_axes(idx, label, x, y, log=True):
    ax = plt.subplot(1, 3, idx)
    ax.scatter(x, y, linewidths=0.1, marker=".")
    if log:
        ax.set_yscale('log')
        ax.set_xscale('log')
    ax.set_xlabel("WBPS")
    ax.set_ylabel(label, rotation=90)
    ax.set_title(f"r={round(np.corrcoef(x, y)[1, 0], 2)} (\N{GREEK CAPITAL LETTER SIGMA}={len(x)})")

In [15]:
scatter_axes(1, "Anno", x1, y1)
scatter_axes(2, "BRAKER3", x2, y2)
scatter_axes(3, "Helixer", x3, y3)
plt.suptitle("Amino acid count correlations")
plt.tight_layout()
plt.savefig(os.path.join(plot_dir, "sman_prot_length_correlations.png"))
plt.show()

In [16]:
scatter_axes(1, "Anno", x1e, y1e, log=False)
scatter_axes(2, "BRAKER3", x2e, y2e, log=False)
scatter_axes(3, "Helixer", x3e, y3e, log=False)
plt.suptitle("Exon count correlations")
plt.tight_layout()
plt.savefig(os.path.join(plot_dir, "sman_exon_count_correlations.png"))
plt.show()

# Assessing merged/split genes

In [17]:
braker_merged, braker_split = suspicious_orthologue_pipeline(og_df, wbps_col, braker_col, species_list, seq_id_map)
anno_merged, anno_split = suspicious_orthologue_pipeline(og_df, wbps_col, anno_col, species_list, seq_id_map)
helixer_merged, helixer_split = suspicious_orthologue_pipeline(og_df, wbps_col, helixer_col, species_list, seq_id_map)

100%|██████████| 9122/9122 [01:20<00:00, 113.98it/s]


In [20]:
num_genes = len(list(braker_species.db.all_features(featuretype="gene")))
print(f"BRAKER3: merged={len(braker_merged)}, split={len(braker_split)}, total={round(100*(len(braker_split) + len(braker_merged)*2)/num_genes, 2)}")
num_genes = len(list(helixer_species.db.all_features(featuretype="gene")))
print(f"Helixer: merged={len(helixer_merged)}, split={len(helixer_split)}, total={round(100*(len(helixer_split) + len(helixer_merged)*2)/num_genes, 2)}")
num_genes = len(list(anno_species.db.all_features(featuretype="gene")))
print(f"Anno: merged={len(anno_merged)}, split={len(anno_split)}, total={round(100*(len(anno_split) + len(anno_merged)*2)/num_genes, 2)}")

BRAKER3: merged=17, split=10, total=0.48
Helixer: merged=161, split=20, total=3.45
Anno: merged=204, split=27, total=3.4


# InterPro accession investigation

### BRAKER3

In [21]:
acc_tally_shared_braker3, acc_tally_missed_braker3, acc_tally_novel_braker3, missed_transcripts = interpro_accession_pipeline(db, og_df, wbps_col, braker_col, interproscan_dir)
_, acc_tally_missed_braker3_unassigned, acc_tally_novel_braker3_unassigned, _ = interpro_accession_pipeline(db, no_og_df, wbps_col, braker_col, os.path.join(interproscan_dir, "unassigned_genes"))
acc_tally_missed_braker3 += acc_tally_missed_braker3_unassigned
acc_tally_novel_braker3 += acc_tally_novel_braker3_unassigned

# Find InterPro accessions occurring with significantly different frequency than in control (acc_tally_shared_braker3)
braker3_novel_results = fisher_exact_for_two_lists_of_accessions(acc_tally_novel_braker3, acc_tally_shared_braker3 + acc_tally_missed_braker3)
braker3_missed_results = fisher_exact_for_two_lists_of_accessions(acc_tally_missed_braker3, acc_tally_shared_braker3 + acc_tally_novel_braker3)

In [22]:
list(interpro_accessions_in_novel_transcripts(flatten_nested_dict(acc_product), acc_tally_novel_braker3, braker3_novel_results, 3))

InterPro accessions occurring with significantly higher frequency in novel transcripts than in shared transcripts:
	IPR037104: Annexin superfamily (3 occurrences, 0 expected)
	IPR003961: Fibronectin type III (3 occurrences, 0 expected)
	IPR036116: Fibronectin type III superfamily (3 occurrences, 0 expected)
	IPR036291: NAD(P)-binding domain superfamily (3 occurrences, 1 expected)
	IPR029071: Ubiquitin-like domain superfamily (3 occurrences, 1 expected)

InterPro accessions that are completely missing from shared transcripts, with high frequency in novel transcripts:
	IPR045609: Domain of unknown function DUF6451 (3 occurrences)



['IPR037104', 'IPR003961', 'IPR036116', 'IPR036291', 'IPR029071', 'IPR045609']

In [23]:
interpro_accessions_in_missed_transcripts(flatten_nested_dict(acc_product), acc_tally_missed_braker3, acc_tally_novel_braker3, braker3_missed_results, braker3_novel_results, min_freq)

InterPro accessions that are completely missing from shared transcripts, with high frequency in missed transcripts:

InterPro accessions occurring with significantly higher frequency in missed transcripts than in shared transcripts:
	IPR041036: Glycoside hydrolase family 5, C-terminal domain (10 occurrences, 0 expected)
	IPR009003: Peptidase S1, PA clan (27 occurrences, 0 expected)
	IPR001254: Serine proteases, trypsin domain (25 occurrences, 0 expected)
	IPR043504: Peptidase S1, PA clan, chymotrypsin-like fold (25 occurrences, 0 expected)
	IPR013780: Glycosyl hydrolase, all-beta (11 occurrences, 1 expected)
	IPR017853: Glycoside hydrolase superfamily (13 occurrences, 1 expected)
	IPR000276: G protein-coupled receptor, rhodopsin-like (23 occurrences, 4 expected)
	IPR017452: GPCR, rhodopsin-like, 7TM (26 occurrences, 4 expected)

InterPro accessions occurring as expected in missed transcripts with high frequency:
	IPR027417: P-loop containing nucleoside triphosphate hydrolase (19 occurr

In [24]:
missed_transcripts_with_significantly_more_frequent_accessions(wbps_species.db, missed_transcripts, acc_tally_missed_braker3, braker3_missed_results, min_freq)

SM_V10_1 - Smp_126730.1 - {'IPR000276', 'IPR017452'}
SM_V10_1 - Smp_170610.1 - {'IPR000276', 'IPR017452'}
SM_V10_1 - Smp_315690.2 - {'IPR013780', 'IPR041036', 'IPR017853'}
SM_V10_1 - Smp_316850.1 - {'IPR000276', 'IPR017452'}
SM_V10_1 - Smp_317470.2 - {'IPR013780', 'IPR041036', 'IPR017853'}
SM_V10_1 - Smp_324100.1 - {'IPR017452'}
SM_V10_1 - Smp_325900.1 - {'IPR017452'}
SM_V10_Z - Smp_162980.1 - {'IPR000276'}
SM_V10_Z - Smp_167870.1 - {'IPR000276', 'IPR017452'}
SM_V10_Z - Smp_204060.1 - {'IPR000276', 'IPR017452'}
SM_V10_Z - Smp_303010.1 - {'IPR013780', 'IPR041036', 'IPR017853'}
SM_V10_Z - Smp_322380.1 - {'IPR017452'}
SM_V10_Z - Smp_323980.1 - {'IPR017452'}
SM_V10_3 - Smp_083940.1 - {'IPR000276', 'IPR017452'}
SM_V10_3 - Smp_091950.1 - {'IPR000276', 'IPR017452'}
SM_V10_3 - Smp_177720.1 - {'IPR000276', 'IPR017452'}
SM_V10_3 - Smp_211260.1 - {'IPR009003'}
SM_V10_3 - Smp_241490.1 - {'IPR000276', 'IPR017452'}
SM_V10_3 - Smp_326640.1 - {'IPR000276', 'IPR017452'}
SM_V10_3 - Smp_333690.1 - {'IPR0

### Helixer

In [25]:
acc_tally_shared_helixer, acc_tally_missed_helixer, acc_tally_novel_helixer, missed_transcripts = interpro_accession_pipeline(db, og_df, wbps_col, helixer_col, interproscan_dir)
_, acc_tally_missed_helixer_unassigned, acc_tally_novel_helixer_unassigned, _ = interpro_accession_pipeline(db, no_og_df, wbps_col, helixer_col, os.path.join(interproscan_dir, "unassigned_genes"))
acc_tally_missed_helixer += acc_tally_missed_helixer_unassigned
acc_tally_novel_helixer += acc_tally_novel_helixer_unassigned

# Find InterPro accessions occurring with significantly different frequency than in control (acc_tally_shared_helixer)
helixer_novel_results = fisher_exact_for_two_lists_of_accessions(acc_tally_novel_helixer, acc_tally_shared_helixer + acc_tally_missed_helixer)
helixer_missed_results = fisher_exact_for_two_lists_of_accessions(acc_tally_missed_helixer, acc_tally_shared_helixer + acc_tally_novel_helixer)

In [26]:
list(interpro_accessions_in_novel_transcripts(flatten_nested_dict(acc_product), acc_tally_novel_helixer, helixer_novel_results, 3))

InterPro accessions occurring with significantly higher frequency in novel transcripts than in shared transcripts:
	IPR036964: Ras guanine-nucleotide exchange factor, catalytic domain superfamily (3 occurrences, 0 expected)
	IPR023578: Ras guanine nucleotide exchange factor domain superfamily (3 occurrences, 0 expected)
	IPR036869: Chaperone J-domain superfamily (5 occurrences, 1 expected)
	IPR001623: DnaJ domain (4 occurrences, 1 expected)
	IPR002347: Short-chain dehydrogenase/reductase SDR (3 occurrences, 1 expected)
	IPR013098: Immunoglobulin I-set (3 occurrences, 1 expected)
	IPR003599: Immunoglobulin subtype (6 occurrences, 1 expected)
	IPR003961: Fibronectin type III (4 occurrences, 1 expected)
	IPR036116: Fibronectin type III superfamily (4 occurrences, 1 expected)
	IPR007110: Immunoglobulin-like domain (7 occurrences, 2 expected)
	IPR003598: Immunoglobulin subtype 2 (4 occurrences, 1 expected)
	IPR036179: Immunoglobulin-like domain superfamily (6 occurrences, 2 expected)
	IPR02

['IPR036964',
 'IPR023578',
 'IPR036869',
 'IPR001623',
 'IPR002347',
 'IPR013098',
 'IPR003599',
 'IPR003961',
 'IPR036116',
 'IPR007110',
 'IPR003598',
 'IPR036179',
 'IPR029071',
 'IPR013783',
 'IPR027417',
 'IPR050951']

In [27]:
interpro_accessions_in_missed_transcripts(flatten_nested_dict(acc_product), acc_tally_missed_helixer, acc_tally_novel_helixer, helixer_missed_results, helixer_novel_results, min_freq=1)

InterPro accessions that are completely missing from shared transcripts, with high frequency in missed transcripts:
	IPR002453: Beta tubulin (7 occurrences)
	IPR013838: Beta tubulin, autoregulation binding site (6 occurrences)
	IPR026183: Taxilin family (4 occurrences)
	IPR001951: Histone H4 (3 occurrences)
	IPR019809: Histone H4, conserved site (3 occurrences)
	IPR035425: CENP-T/Histone H4, histone fold (3 occurrences)
	IPR000558: Histone H2B (3 occurrences)
	IPR001971: Small ribosomal subunit protein uS11 (2 occurrences)
	IPR001147: Large ribosomal subunit protein eL21 (2 occurrences)
	IPR018259: Large ribosomal subunit protein eL21, conserved site (2 occurrences)
	IPR036948: Large ribosomal subunit protein eL21 superfamily (2 occurrences)
	IPR005336: Mitochondrial pyruvate carrier (2 occurrences)
	IPR029040: RNA polymerase subunit RPABC4/transcription elongation factor Spt4 (2 occurrences)
	IPR011331: Large ribosomal subunit protein eL37/eL43 (2 occurrences)
	IPR000892: Small riboso

### Anno

In [29]:
acc_tally_shared_anno, acc_tally_missed_anno, acc_tally_novel_anno, missed_transcripts = interpro_accession_pipeline(db, og_df, wbps_col, anno_col, interproscan_dir)
_, acc_tally_missed_anno_unassigned, acc_tally_novel_anno_unassigned, _ = interpro_accession_pipeline(db, no_og_df, wbps_col, anno_col, os.path.join(interproscan_dir, "unassigned_genes"))
acc_tally_missed_anno += acc_tally_missed_anno_unassigned
acc_tally_novel_anno += acc_tally_novel_anno_unassigned

# Find InterPro accessions occurring with significantly different frequency than in control (acc_tally_shared_anno)
anno_novel_results = fisher_exact_for_two_lists_of_accessions(acc_tally_novel_anno, acc_tally_shared_anno + acc_tally_missed_anno)
anno_missed_results = fisher_exact_for_two_lists_of_accessions(acc_tally_missed_anno, acc_tally_shared_anno + acc_tally_novel_anno)

In [30]:
list(interpro_accessions_in_novel_transcripts(flatten_nested_dict(acc_product), acc_tally_novel_anno, anno_novel_results, min_freq=1))

InterPro accessions occurring with significantly higher frequency in novel transcripts than in shared transcripts:
	IPR001995: Peptidase A2A, retrovirus, catalytic (2 occurrences, 0 expected)
	IPR043128: Reverse transcriptase/Diguanylate cyclase domain (5 occurrences, 0 expected)
	IPR000477: Reverse transcriptase domain (4 occurrences, 0 expected)
	IPR027080: Protein Unc-13 (2 occurrences, 0 expected)
	IPR007081: RNA polymerase Rpb1, domain 5 (2 occurrences, 0 expected)
	IPR043502: DNA/RNA polymerase superfamily (6 occurrences, 0 expected)
	IPR005479: Carbamoyl-phosphate synthetase large subunit-like, ATP-binding domain (2 occurrences, 0 expected)
	IPR033749: Polyprenyl synthetase, conserved site (2 occurrences, 0 expected)
	IPR045867: DNA-directed RNA polymerase, subunit beta-prime (2 occurrences, 0 expected)
	IPR038207: DIX domain superfamily (2 occurrences, 0 expected)
	IPR001158: DIX domain (2 occurrences, 0 expected)
	IPR016185: Pre-ATP-grasp domain superfamily (2 occurrences, 0 e

['IPR001995',
 'IPR043128',
 'IPR000477',
 'IPR027080',
 'IPR007081',
 'IPR043502',
 'IPR005479',
 'IPR033749',
 'IPR045867',
 'IPR038207',
 'IPR001158',
 'IPR016185',
 'IPR043154',
 'IPR036045',
 'IPR008949',
 'IPR048684',
 'IPR036430',
 'IPR021109',
 'IPR011761',
 'IPR001300',
 'IPR000306',
 'IPR037272',
 'IPR000175',
 'IPR036880',
 'IPR002223',
 'IPR002126',
 'IPR023298',
 'IPR015919',
 'IPR003599',
 'IPR020894',
 'IPR018490',
 'IPR000595',
 'IPR036179',
 'IPR007110',
 'IPR000387',
 'IPR036259',
 'IPR036770',
 'IPR013783',
 'IPR002110',
 'IPR050174',
 'IPR051877',
 'IPR050098',
 'IPR050599',
 'IPR050951',
 'IPR050327',
 'IPR050122',
 'IPR051131',
 'IPR050382',
 'IPR050302',
 'IPR050216',
 'IPR052993',
 'IPR050927',
 'IPR051835',
 'IPR050217',
 'IPR031703',
 'IPR036906',
 'IPR008218',
 'IPR019186',
 'IPR019269',
 'IPR050964',
 'IPR053025',
 'IPR028119',
 'IPR002056',
 'IPR023392',
 'IPR022422',
 'IPR041577',
 'IPR051165',
 'IPR051783',
 'IPR015816',
 'IPR050733',
 'IPR050668',
 'IPR0

In [31]:
interpro_accessions_in_missed_transcripts(flatten_nested_dict(acc_product), acc_tally_missed_anno, acc_tally_novel_anno, anno_missed_results, anno_novel_results, min_freq)

InterPro accessions that are completely missing from shared transcripts, with high frequency in missed transcripts:

InterPro accessions occurring with significantly higher frequency in missed transcripts than in shared transcripts:
	IPR009003: Peptidase S1, PA clan (22 occurrences, 0 expected)
	IPR001254: Serine proteases, trypsin domain (20 occurrences, 0 expected)
	IPR043504: Peptidase S1, PA clan, chymotrypsin-like fold (20 occurrences, 0 expected)

InterPro accessions occurring as expected in missed transcripts with high frequency:

InterPro accessions occurring less frequently in missed transcripts than expected:
	IPR000719: Protein kinase domain (1 occurrences, 7 expected)
	IPR011009: Protein kinase-like domain superfamily (2 occurrences, 8 expected)
	IPR027417: P-loop containing nucleoside triphosphate hydrolase (6 occurrences, 14 expected)



### General

Find accessions that are significantly commonly missed by **all** tools

In [32]:
acc_tally_no_tool, acc_tally_one_plus_tool_shared, acc_tally_one_plus_tool_novel = interpro_accession_pipeline_all_tools(
    wbps_species,
    og_df,
    wbps_col,
    [braker_col, helixer_col, anno_col],
    interproscan_dir
)

acc_tally_no_tool_unassigned, _, acc_tally_one_plus_tool_novel_unassigned = interpro_accession_pipeline_all_tools(
    wbps_species,
    no_og_df,
    wbps_col,
    [braker_col, helixer_col, anno_col],
    os.path.join(interproscan_dir, "unassigned_genes")
)
acc_tally_no_tool += acc_tally_no_tool_unassigned
acc_tally_one_plus_tool_novel += acc_tally_one_plus_tool_novel_unassigned

no_tools_results = fisher_exact_for_two_lists_of_accessions(acc_tally_no_tool, acc_tally_one_plus_tool_shared + acc_tally_one_plus_tool_novel)
only_tools_results = fisher_exact_for_two_lists_of_accessions(acc_tally_one_plus_tool_novel, acc_tally_one_plus_tool_shared + acc_tally_no_tool)

In [36]:
def is_AP(item):
    return item[0] in ASPARTIC_PEPTIDASE_SUPERFAMILY

def filter_results_for_AP(results):
    output = {}
    for k, v in results.items():
        try:
            output[k] = dict(filter(is_AP, v.items()))
        except AttributeError:
            output[k] = list(filter(is_AP, v))
    return output

only_tools_results_AP = filter_results_for_AP(only_tools_results)
no_tools_results_AP = filter_results_for_AP(no_tools_results)
print(only_tools_results_AP)
print(no_tools_results_AP)

{'more_frequent': {'IPR021109': [11.860385731353473, 4.709663083007144e-07], 'IPR001995': [56.77794793261868, 8.927868241734657e-05]}, 'as_expected': {}, 'less_frequent': {}, 'not_occurring': []}
{'more_frequent': {}, 'as_expected': {}, 'less_frequent': {}, 'not_occurring': []}


In [43]:
overrepd_accs = list(interpro_accessions_in_novel_transcripts(flatten_nested_dict(acc_product), acc_tally_one_plus_tool_novel, only_tools_results, min_freq=1))

InterPro accessions occurring with significantly higher frequency in novel transcripts than in shared transcripts:
	IPR001995: Peptidase A2A, retrovirus, catalytic (4 occurrences, 0 expected)
	IPR000477: Reverse transcriptase domain (13 occurrences, 0 expected)
	IPR043128: Reverse transcriptase/Diguanylate cyclase domain (9 occurrences, 0 expected)
	IPR029008: ER membrane protein complex subunit 6-like (3 occurrences, 0 expected)
	IPR013209: Lipin/Ned1/Smp2 (LNS2) (3 occurrences, 0 expected)
	IPR007651: Lipin, N-terminal (3 occurrences, 0 expected)
	IPR006139: D-isomer specific 2-hydroxyacid dehydrogenase, catalytic domain (3 occurrences, 0 expected)
	IPR006140: D-isomer specific 2-hydroxyacid dehydrogenase, NAD-binding domain (3 occurrences, 0 expected)
	IPR023509: D-aminoacyl-tRNA deacylase-like superfamily (3 occurrences, 0 expected)
	IPR003732: D-aminoacyl-tRNA deacylase DTD (3 occurrences, 0 expected)
	IPR011387: Translation initiation factor 2A (2 occurrences, 0 expected)
	IPR011

There don't appear to be any TEs amongst the overrepresented "homologous superfamily" or "family" accessions

In [44]:
print(len(overrepd_accs))
with open("data/transposable_elements/transposable_element_iprs.json", "r") as f:
    te_accs = json.load(f)
set(te_accs.values()).intersection(overrepd_accs)

265


{'IPR000477', 'IPR001584', 'IPR002156', 'IPR024445'}

In [45]:
anno_prefix = "transcript_ENSSMAT"
braker_prefix = "g"
helixer_prefix = "Schistosoma_mansoni_SM_V10_"
non_te_unique_transcripts = set()
te_unique_transcripts = set()

for acc in tqdm(overrepd_accs, total=len(overrepd_accs)):
    tool_transcript_counts = [count_transcripts_with_accession(acc, p, interproscan_dir) for p in (anno_prefix, braker_prefix, helixer_prefix)]
    unique_transcripts = flatten_list_to_set(list(map(itemgetter(1), tool_transcript_counts)))
    print(f"{acc}: {sum(map(itemgetter(0), tool_transcript_counts))} ({len(unique_transcripts)} unique)")
    if acc in te_accs:
        te_unique_transcripts.update(unique_transcripts)
    else:
        non_te_unique_transcripts.update(unique_transcripts)

  0%|          | 1/265 [00:01<04:31,  1.03s/it]

IPR001995: 4 (4 unique)


  1%|          | 2/265 [00:01<04:19,  1.01it/s]

IPR000477: 13 (13 unique)


  1%|          | 3/265 [00:02<04:18,  1.01it/s]

...

IPR011322: 0 (0 unique)


100%|█████████▉| 264/265 [04:22<00:00,  1.07it/s]

IPR000077: 0 (0 unique)


100%|██████████| 265/265 [04:23<00:00,  1.01it/s]

IPR023626: 0 (0 unique)


204 unique novel genes associated with overrepresented IPRs, none of which are associated explicitly with TEs.

- 85 Anno
- 49 BRAKER3
- 70 Helixer

In [46]:
print(len(non_te_unique_transcripts))
print(len([t for t in non_te_unique_transcripts if t.startswith(anno_prefix)]))
print(len([t for t in non_te_unique_transcripts if t.startswith(braker_prefix)]))
print(len([t for t in non_te_unique_transcripts if t.startswith(helixer_prefix)]))

# for t in non_te_unique_transcripts:
#     print(t)

235
93
65
77


In [47]:
te_unique_transcripts

set()

In [50]:
interpro_accessions_frequently_missed_by_all_tools(flatten_nested_dict(acc_product), acc_tally_no_tool, no_tools_results, min_freq=5)

InterPro accessions occurring with significantly higher frequency in transcripts that were missed by all tools, than in transcripts shared by at least 1 tool:
	IPR001254: Serine proteases, trypsin domain (20 occurrences, 0 expected)
	IPR043504: Peptidase S1, PA clan, chymotrypsin-like fold (20 occurrences, 0 expected)
	IPR009003: Peptidase S1, PA clan (21 occurrences, 0 expected)
	IPR007125: Histone H2A/H2B/H3 (5 occurrences, 0 expected)
	IPR009072: Histone-fold (8 occurrences, 0 expected)

InterPro accessions that are completely missing from transcripts shared by at least 1 tool, but present in transcripts that were missed by all tools:

InterPro accessions occurring as expected in transcripts that were missed by all tools.


Find accessions that are significantly commonly missed by each tool.

In [51]:
interpro_accessions_frequently_missed_by_each_tool(
    flatten_nested_dict(acc_product),
    {
        "BRAKER3": braker3_missed_results,
        "Helixer": helixer_missed_results,
        "Anno": anno_missed_results
    },
    {
        "BRAKER3": acc_tally_missed_braker3,
        "Helixer": acc_tally_missed_helixer,
        "Anno": acc_tally_missed_anno
    }
)

IPR015898: G-protein gamma-like domain
40.6 times more likely than with BRAKER3 (2 occurrences, 0 expected)
46.25 times more likely than with Helixer (2 occurrences, 0 expected)
69.27 times more likely than with Anno (2 occurrences, 0 expected)
~~~~~~~~~~
IPR036284: G-protein gamma-like domain superfamily
40.6 times more likely than with BRAKER3 (2 occurrences, 0 expected)
46.25 times more likely than with Helixer (2 occurrences, 0 expected)
69.27 times more likely than with Anno (2 occurrences, 0 expected)
~~~~~~~~~~
IPR002119: Histone H2A
16.26 times more likely than with BRAKER3 (4 occurrences, 0 expected)
69.61 times more likely than with Helixer (6 occurrences, 0 expected)
20.81 times more likely than with Anno (3 occurrences, 0 expected)
~~~~~~~~~~
IPR032458: Histone H2A conserved site
15.24 times more likely than with BRAKER3 (3 occurrences, 0 expected)
115.92 times more likely than with Helixer (5 occurrences, 0 expected)
17.32 times more likely than with Anno (2 occurrences, 0

In [53]:
import re
MIN_WORD_LEN = 5
tools_missed_results = {
    "BRAKER3": braker3_missed_results,
    "Helixer": helixer_missed_results,
    "Anno": anno_missed_results
}
tools_novel_results = {
    "BRAKER3": braker3_novel_results,
    "Helixer": helixer_novel_results,
    "Anno": anno_novel_results
}
acc_tally_missed_tools = {
    "BRAKER3": acc_tally_missed_braker3,
    "Helixer": acc_tally_missed_helixer,
    "Anno": acc_tally_missed_anno
}
acc_tally_novel_tools = {
    "BRAKER3": acc_tally_novel_braker3,
    "Helixer": acc_tally_novel_helixer,
    "Anno": acc_tally_novel_anno
}
seen_words = set()
all_words = set()
for acc in set.intersection(*[set(results["more_frequent"].keys()) for results in tools_missed_results.values()]):
    prod = flatten_nested_dict(acc_product)[acc]
    words = [w for w in re.split(r'\W', prod.lower()) if w and len(w) >= MIN_WORD_LEN]
    all_words.update(words)
    # Check if words occur more frequently in novel-only accessions
    for novel_acc in set.intersection(*[set(results["more_frequent"].keys()) for results in tools_novel_results.values()]):
        novel_prod = flatten_nested_dict(acc_product)[novel_acc]
        for w in words:
            if w in novel_prod.lower():
                print(f"{w} appears in {novel_acc}: {novel_prod}")
                seen_words.add(w)

In [54]:
all_words.difference(seen_words)

{'chymotrypsin',
 'conserved',
 'domain',
 'gamma',
 'histone',
 'patch',
 'peptidase',
 'proteases',
 'protein',
 'serine',
 'superfamily',
 'trypsin',
 'uncharacterised',
 'upf0506'}

# Novel orthologues

In [55]:
print("Novel transcripts (% of total predicted by tool)")

shared_braker_genes = len(og_df[~og_df[wbps_col].isna() & ~og_df[braker_col].isna()][braker_col].str.split(",").explode().unique())
novel_braker_orths = len(og_df[og_df[wbps_col].isna() & ~og_df[braker_col].isna()][braker_col].str.split(",").explode().unique())
novel_braker_ungs = len(no_og_df[no_og_df[wbps_col].isna() & ~no_og_df[braker_col].isna()][braker_col].str.split(",").explode().unique())
assert shared_braker_genes + novel_braker_orths + novel_braker_ungs == len(braker_species.all_transcript_ids)
print(f"BRAKER3: {novel_braker_orths + novel_braker_ungs} ({round(100*(novel_braker_orths + novel_braker_ungs)/len(braker_species.all_transcript_ids), 2)}%)")


shared_helixer_genes = len(og_df[~og_df[wbps_col].isna() & ~og_df[helixer_col].isna()][helixer_col].str.split(",").explode().unique())
novel_helixer_orths = len(og_df[og_df[wbps_col].isna() & ~og_df[helixer_col].isna()][helixer_col].str.split(",").explode().unique())
novel_helixer_ungs = len(no_og_df[no_og_df[wbps_col].isna() & ~no_og_df[helixer_col].isna()][helixer_col].str.split(",").explode().unique())
assert shared_helixer_genes + novel_helixer_orths + novel_helixer_ungs == len(helixer_species.all_transcript_ids)
print(f"Helixer: {novel_helixer_orths + novel_helixer_ungs} ({round(100*(novel_helixer_orths + novel_helixer_ungs)/len(helixer_species.all_transcript_ids), 2)}%)")

shared_anno_genes = len(og_df[~og_df[wbps_col].isna() & ~og_df[anno_col].isna()][anno_col].str.split(",").explode().unique())
novel_anno_orths = len(og_df[og_df[wbps_col].isna() & ~og_df[anno_col].isna()][anno_col].str.split(",").explode().unique())
novel_anno_ungs = len(no_og_df[no_og_df[wbps_col].isna() & ~no_og_df[anno_col].isna()][anno_col].str.split(",").explode().unique())
assert shared_anno_genes + novel_anno_orths + novel_anno_ungs == len(anno_species.all_transcript_ids)
print(f"Anno: {novel_anno_orths + novel_anno_ungs} ({round(100*(novel_anno_orths + novel_anno_ungs)/len(anno_species.all_transcript_ids), 2)}%)")


Novel transcripts (% of total predicted by tool)
BRAKER3: 508 (5.59%)
Helixer: 1164 (11.76%)
Anno: 2720 (21.25%)


In [4]:
## Run once to populate
# novel_orthologue_pipeline(no_og_df, wbps_col, anno_col, species_list, out_dir="data/novel_orthologue_sequences/sman/anno/unassigned_genes/")
# novel_orthologue_pipeline(no_og_df, wbps_col, braker_col, species_list, out_dir="data/novel_orthologue_sequences/sman/braker3/unassigned_genes/")
# novel_orthologue_pipeline(no_og_df, wbps_col, helixer_col, species_list, out_dir="data/novel_orthologue_sequences/sman/helixer/unassigned_genes/")

100%|██████████| 2437/2437 [00:06<00:00, 358.46it/s] 


In [56]:
# novel_orthologue_pipeline(og_df, wbps_col, anno_col, species_list)
# novel_orthologue_pipeline(og_df, wbps_col, braker_col, species_list)
# novel_orthologue_pipeline(og_df, wbps_col, helixer_col, species_list)
anno_esm_means = extract_esm_means("data/from_MARS/Sman_esm_pLDDTs_anno.txt").values()
braker3_esm_means = extract_esm_means("data/from_MARS/Sman_esm_pLDDTs_braker3.txt").values()
helixer_esm_means = extract_esm_means("data/from_MARS/Sman_esm_pLDDTs_helixer.txt").values()

print(statistics.mean(map(float, anno_esm_means)))
print(statistics.mean(map(float, braker3_esm_means)))
print(statistics.mean(map(float, helixer_esm_means)))

def boxplot_axes(idx, label, data, ylim=(0, 100)):
    ax = plt.subplot(1, 4, idx)
    ax.boxplot(data)
    if ylim:
        ax.set_ylim(top=ylim[1], bottom=ylim[0])
    ax.grid(which="both")
    ax.set_title(label.upper())
    ax.set_xticklabels([])
    ax.set_xlabel(round(statistics.mean(data), 2))

boxplot_axes(idx=1, label="anno", data=list(map(float, anno_esm_means)))
boxplot_axes(idx=2, label="braker3", data=list(map(float, braker3_esm_means)))
boxplot_axes(idx=3, label="helixer", data=list(map(float, helixer_esm_means)))
plt.show()


# anno_esm_means

54.20466926070039
57.278228782287826
54.64353233830846


In [57]:
import pandas as pd
cols = (
    "fn",
    "mean",
    "median",
    "stdev",
    "var",
    "max",
    "min",
    "perc_confident"
)
df = pd.read_csv("data/from_MARS/pLDDT_sman.csv", names=cols)
print(f"Mean of means: {df['mean'].mean()}")
print(f"% that are \"Confident\": {100*df[df['mean'] >= 70].shape[0]/df.shape[0]}")

# df.count()

Mean of means: 70.15563125399191
% that are "Confident": 54.939322972109856


In [58]:
print(f"% BRAKER3 that are \"Confident\": {100*len([p for p in braker3_esm_means if float(p) >= 70]) / len(braker3_esm_means)}")
print(f"% Anno that are \"Confident\": {100*len([p for p in anno_esm_means if float(p) >= 70]) / len(anno_esm_means)}")
print(f"% Helixer that are \"Confident\": {100*len([p for p in helixer_esm_means if float(p) >= 70]) / len(helixer_esm_means)}")

% BRAKER3 that are "Confident": 31.73431734317343
% Anno that are "Confident": 24.90272373540856
% Helixer that are "Confident": 21.890547263681594


In [59]:

df_anno = pd.read_csv("data/from_MARS/pLDDT_sman_anno.csv", names=cols)
df_braker3 = pd.read_csv("data/from_MARS/pLDDT_sman_braker3.csv", names=cols)
df_helixer = pd.read_csv("data/from_MARS/pLDDT_sman_helixer.csv", names=cols)

In [60]:
print(f"Mean of WBPS % that are \"Confident\" residues: {df['perc_confident'].mean()}")
print(f"Mean of BRAKER3 % that are \"Confident\" residues: {df_braker3['perc_confident'].mean()}")
print(f"Mean of Anno % that are \"Confident\" residues: {df_anno['perc_confident'].mean()}")
print(f"Mean of Helixer % that are \"Confident\" residues: {df_helixer['perc_confident'].mean()}")


Mean of WBPS % that are "Confident" residues: 57.85118160527997
Mean of BRAKER3 % that are "Confident" residues: 35.38376383763838
Mean of Anno % that are "Confident" residues: 30.747081712062258
Mean of Helixer % that are "Confident" residues: 30.46153846153846


In [61]:
df_anno[(df_anno["mean"] > 50) & (df_anno["perc_confident"] == 0)].sort_values("stdev")

,fn,mean,median,stdev,var,max,min,perc_confident
319,transcript:ENSSMAT00000005813.pdb,56,57,6,36,65,36,0
186,transcript:ENSSMAT00000008498.pdb,55,55,7,46,69,38,0
215,transcript:ENSSMAT00000034886.pdb,56,58,7,44,65,42,0
80,transcript:ENSSMAT00000012469.pdb,54,54,8,65,67,36,0
47,transcript:ENSSMAT00000034121.pdb,53,55,9,73,67,25,0
146,transcript:ENSSMAT00000026481.pdb,51,54,10,100,66,34,0
242,transcript:ENSSMAT00000028062.pdb,51,47,10,93,69,34,0
362,transcript:ENSSMAT00000030008.pdb,51,52,11,113,69,30,0


In [32]:
df_braker3[(df_braker3["mean"] > 50) & (df_braker3["perc_confident"] == 0)].sort_values("stdev")

,fn,mean,median,stdev,var,max,min,perc_confident
46,g2824.t1.pdb,55,55,3,12,62,49,0
15,g2879.t1.pdb,57,57,4,17,68,50,0
138,g2535.t1.pdb,61,62,5,25,70,49,0
117,g5842.t1.pdb,54,55,7,45,65,32,0
54,g1749.t1.pdb,51,52,11,113,69,30,0


In [33]:
df_helixer[(df_helixer["mean"] < 60) & (df_helixer["perc_confident"] > 40)]

,fn,mean,median,stdev,var,max,min,perc_confident
363,Schistosoma_mansoni_SM_V10_Z_001959.1.pdb,58,61,22,474,90,22,41


In [62]:
df_helixer[(df_helixer["mean"] > 50) & (df_helixer["perc_confident"] == 0)].sort_values("stdev")

,fn,mean,median,stdev,var,max,min,perc_confident
325,Schistosoma_mansoni_SM_V10_4_001163.1.pdb,53,53,3,10,59,46,0
237,Schistosoma_mansoni_SM_V10_6_000170.1.pdb,59,58,5,20,67,52,0
282,Schistosoma_mansoni_SM_V10_2_000542.1.pdb,51,50,6,36,65,41,0
304,Schistosoma_mansoni_SM_V10_2_000788.1.pdb,51,49,6,36,62,41,0
378,Schistosoma_mansoni_SM_V10_5_000309.1.pdb,51,53,6,35,61,36,0
21,Schistosoma_mansoni_SM_V10_1_000685.1.pdb,53,53,7,55,69,42,0
45,Schistosoma_mansoni_SM_V10_4_000066.1.pdb,57,59,8,69,69,19,0
48,Schistosoma_mansoni_SM_V10_4_000006.1.pdb,51,48,8,62,64,39,0
316,Schistosoma_mansoni_SM_V10_2_001157.1.pdb,51,50,8,59,65,35,0
332,Schistosoma_mansoni_SM_V10_2_000183.1.pdb,53,55,8,60,66,35,0
